# Le choixpeau magique
## Fusion de tables
### Caractéristiques des élèves de Poudlard

Le fichier initial, contenant les personnages du roman _Harry Potter_ vient du [site Kaggle](https://www.kaggle.com/gulsahdemiryurek/harry-potter-dataset).

En l'état, ce fichier ne permet pas de créer notre Choixpeau magique car il ne contient pas les __caractéristiques des personnages__ (Courage, Ambition, Intelligence, Tendance à la bonté), essentielles à 
la création d'un __profil par personnage__.

Il faut donc créer puis ajouter ces informations aux données du fichier initial. Le __fichier Caracteristiques_des_persos.csv__ contient ces nouvelles données sous la forme...

```
Name;Courage;Ambition;Intelligence;Good
Harry James Potter;9;4;5;9
Ronald Bilius Weasley;8;6;5;9
Hermione Jean Granger;9;6;9;9
etc...
```

> __Remarques :__ 
- ayant pour objectif de créer un Choixpeau magique, seuls les élèves ou anciens élèves de Poudlard nous intéressent. L'ajout de ces données n'a donc été fait que sur ces personnages.
- quelques anciens élèves étant difficiles à cerner, ils ne seront pas non plus conservés dans la base finale.

### Création des deux tables

Dans un premier temps, il faut __créer deux tables à partir des deux fichiers CSV__ à notre disposition : __Characters.csv__ et __Caracteristiques_des_persos.csv__.

> __Remarques :__ 
- ici encore, le choix technique pour créer les tables se porte classiquement sur un tableau de dictionnaires.
- a ce stade, il vous est permis d'utiliser le module `csv`.

#### Création de la table `characters_tab` à partir du fichier Characters.csv

A vous de jouer...

In [ ]:
import csv

with open("Characters.csv", mode='r', encoding='utf-8') as f:
    reader = csv.DictReader(f, delimiter=';')
    characters_tab = [{key : value for key, value in element.items()} for element in reader]

print(characters_tab)

> __Remarque :__ il arrive que certains caractères indésirables posent un problème d'affichage, comme les espaces insécables `\xa0`. Un nettoyage avec la methode [`.replace('\xa0', ' ')`](https://www.w3schools.com/python/ref_string_replace.asp) au moment de la création de la table peut résoudre de souci.

Au besoin, faire ce nettoyage ci-dessous :

In [ ]:
with open("Characters.csv", mode='r', encoding='utf-8') as f:
    reader = csv.DictReader(f, delimiter=';')
    characters_tab = [{key : value.replace('\xa0', ' ') for key, value in element.items()} for element in reader]

print(characters_tab)

#### Création de la table `characteristics_tab` à partir du fichier Caracteristiques_des_persos.csv

In [ ]:
import csv

with open("Caracteristiques_des_persos.csv", mode='r', encoding='utf-8') as f:
    reader = csv.DictReader(f, delimiter=';')
    characteristics_tab = [{key : value for key, value in element.items()} for element in reader]

print(characteristics_tab)

### Jointure des deux tables

Il faut maintenant __créer une nouvelle table `poudlard_characters`__ en effectuant une jointure entre les deux tables précédentes.

Il faut donc...

- __conserver uniquement les élèves de Poudlard__ dont on dispose des caractéristiques.
- __ajouter aux enregistrements de ces élèves tous les attributs__ à notre disposition dans le fichier d'origine (Characters.csv).

A vous de jouer...

In [ ]:
poudlard_characters = []

for poudlard_character in characteristics_tab:
    for kaggle_character in characters_tab:
        if poudlard_character['Name'] == kaggle_character['Name']:
            poudlard_character.update(kaggle_character)
            poudlard_characters.append(poudlard_character)

print(poudlard_characters)

## Indexations pour les besoins du Choixpeau magique

> __Remarque :__ dans le cadre du projet "Choixpeau magique", l'indexation n'est pas forcément utile. De plus, l'indexation rend le projet plus complexe. __Cette indexation est donc à réserver aux élèves qui feront le bonus sur la validation croisée__ car, dans ce cas, l'indexation apporte un réel gain de performances.

Le Choixpeau magique nécessite de profiler chaque élève à l'aide des caractéristiques que nous venons de fusionner. 

Par souci d'efficacité, nous allons donc...

- créer un gros __index nommé `index_id`, liant chaque élève à l'ensemble de ses attributs__.
- créer un petit __index nommé `index_id_characteristics`, liant chaque élève à ses caractéristiques__ (Courage, Ambition, Intelligence, Tendance au bien).

> __Remarques :__
- __on ne gardera que le descripteur `Id` pour désigner un élève__.
- __un identifiant unique__ par personnage, comme `Id`, évite les éventuels __problèmes en cas d'homonymie__.

### Indexation de l'ensemble de la table, par identifiant

Afin de retrouver facilement et rapidement un attribut quelconque à partir de l'identifiant d'un élève, créer un index sous la forme :

    index_id = {1: {'Name': 'Harry James Potter', 'Courage': '9', 'Ambition': '4', 'Intelligence': '5', 'Good': '9', 'Id': '1', 'Gender': 'Male', 'Job': 'Student', 'House': 'Gryffindor', 'Wand': '11"  Holly  phoenix feather', 'Patronus': 'Stag', 'Species': 'Human', 'Blood status': 'Half-blood', 'Hair colour': 'Black', 'Eye colour': 'Bright green', 'Loyalty': "Albus Dumbledore | Dumbledore's Army | Order of the Phoenix | Hogwarts School of Witchcraft and Wizardry", 'Skills': 'Parseltongue| Defence Against the Dark Arts | Seeker', 'Birth': '31 July 1980', 'Death': ''},
                2: {'Name': 'Ronald Bilius Weasley', 'Courage': '8', 'Ambition': '6', 'Intelligence': '5', 'Good': '9', 'Id': '2', 'Gender': 'Male', 'Job': 'Student', 'House': 'Gryffindor', 'Wand': '12" Ash unicorn tail hair ', 'Patronus': 'Jack Russell terrier', 'Species': 'Human', 'Blood status': 'Pure-blood', 'Hair colour': 'Red', 'Eye colour': 'Blue', 'Loyalty': "Dumbledore's Army | Order of the Phoenix | Hogwarts School of Witchcraft and Wizardry", 'Skills': 'Wizard chess | Quidditch goalkeeping', 'Birth': '1 March 1980', 'Death': ''},
                 3: ...}

> __Remarque :__ les __identifiants uniques__ (ex : numéro de sécurité sociale, ISBN pour les livres,...) sont le plus souvent __des entiers__. On convertira donc en entier toutes les clés de cet index (voir modèle ci-dessus).

A vous de jouer...

In [ ]:
index_id = {int(character['Id']): character for character in poudlard_characters}

print(index_id)

### Indexation des 4 caractéristiques par identifiant

Afin de retrouver facilement et rapidement ses 4 caractéristiques à partir de l'identifiant d'un élève, il faut maintenant créer un index dédié à ces recherches.

> __Remarques :__
- on utilisera un __tuple__ sous la forme (valeur de Courage, valeur d'Ambition, valeur d'Intelligence, valeur de tendance au bien) pour optimiser l'espace et l'efficacité algo.
- toutes les __valeurs numériques seront des entiers__.

L'index devra donc être sous la forme :

`index_id_characteristics = {1: (9, 4, 5, 9), 2: (8, 6, 5, 9), ... }`



In [ ]:
index_id_characteristics = {int(character['Id']): 
                            (int(character['Courage']),
                             int(character['Ambition']),
                             int(character['Intelligence']),
                             int(character['Good'])) 
                            for character in poudlard_characters}

print(index_id_characteristics)

Nous savons maintenant comment construire les __structures de données adaptées à notre projet du Choixpeau magique__.

---
[![Licence CC BY NC SA](https://licensebuttons.net/l/by-nc-sa/3.0/88x31.png "licence Creative Commons CC BY-NC-SA")](http://creativecommons.org/licenses/by-nc-sa/3.0/fr/)
<p style="text-align: center;">Auteur : David Landry, Lycée Clemenceau - Nantes</p>